In [5]:
from pprint import pprint
from DbConnector import DbConnector
from tabulate import tabulate

In [75]:
class GeolifeQueries:

    def __init__(self):
        # Set up the database connection
        self.connection = DbConnector()
        self.client = self.connection.client
        self.db = self.connection.db

    def print_tables(self):
        print("Users")
        first_ten_users= list(self.db.User.find().limit(10))

        table_data = []

        for document in first_ten_users:
            table_data.append([
                document.get('_id'),      
                document.get('has_labels')
            ])
        headers = ['ID', 'Has Labels']
        print(tabulate(table_data, headers=headers, tablefmt='fancy_grid'))
        print("\n")

        print("Activities")
        first_ten_activities = list(self.db.Activity.find().limit(10))

        table_data = []

        for document in first_ten_activities:
            table_data.append([
                document.get('_id'),              
                document.get('user_id'), 
                document.get('transportation_mode'), 
                document.get('start_date_time').strftime("%Y-%m-%d %H:%M:%S"),
                document.get('end_date_time').strftime("%Y-%m-%d %H:%M:%S")
                    ])
        headers = ['ID', 'UserID', 'Transportation Mode', 'Start Date Time', 'End Date Time']
        print(tabulate(table_data, headers=headers, tablefmt='fancy_grid'))
        print("\n")

        print("Trackpoints")
        first_ten_trackpoints = list(self.db.TrackPoint.find().limit(10))

        table_data = []

        for document in first_ten_trackpoints:
            table_data.append([
                document.get('_id'),              
                document.get('lat'),              
                document.get('lon'),              
                document.get('altitude'),         
                document.get('date_days'),        
                document.get('date_time') 
            ])
        headers = ['ID', 'Latitude', 'Longitude', 'Altitude', 'Date Days', 'Date Time']
        print(tabulate(table_data, headers=headers, tablefmt='fancy_grid'))
        print("\n")

    def count_users_activities_trackpoints(self):
        """Task 1: Count users, activities, and trackpoints."""
        print("Task 1")
        num_users = self.db.User.count_documents({})
        num_activities = self.db.Activity.count_documents({})
        num_trackpoints = self.db.TrackPoint.count_documents({})
        print(f"Users: {num_users}\nActivities: {num_activities}\nTrackpoints: {num_trackpoints}")

    def average_activities_per_user(self):
        """Task 2: Find the average number of activities per user."""
        print("Task 2")
        num_activities = self.db.Activity.count_documents({})
        num_users = self.db.User.count_documents({})
        avg_activities = num_activities / num_users if num_users != 0 else 0
        print(f"Average activities per user: {avg_activities:.2f}")

    def top_20_users_by_activities(self):
        """Task 3: Find the top 20 users with the highest number of activities."""
        print("Task 3")
        pipeline = [
            {"$group": {"_id": "$user_id", "activity_count": {"$sum": 1}}},
            {"$sort": {"activity_count": -1}},
            {"$limit": 20}
        ]
        top_users = self.db.Activity.aggregate(pipeline)

        # Prepare data for tabulate
        table_data = []
        for user in top_users:
            table_data.append([user['_id'], user['activity_count']])
        
        # Print the table with the fancy_grid format
        print(tabulate(table_data, headers=["ID", "Activity Count"], tablefmt="grid"))
    
    def users_who_took_taxi(self):
        """Task 4: Find and print all users who have taken a taxi."""
        print("Task 4")
        
        # Query to find documents where transportation_mode is "taxi"
        documents = self.db.Activity.find({"transportation_mode": "taxi"}, {"user_id": 1, "_id": 0})
        
        # Set to hold unique user IDs
        taxi_users = set()
        
        # Loop through the documents and add user_ids to the set
        for doc in documents:
            taxi_users.add(doc["user_id"])

        sorted_taxi_users = sorted(taxi_users)
        
        # Print the unique user_ids
        print(tabulate([[user_id] for user_id in sorted_taxi_users], headers=["User ID"], tablefmt="fancy_grid"))


    def count_transportation_modes(self):
        """Task 5: Count activities with each transportation mode."""
        print("Task 5")
        pipeline = [
            {"$match": {"transportation_mode": {"$ne": None}}},
            {"$group": {"_id": "$transportation_mode", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]
        transport_modes = self.db.Activity.aggregate(pipeline)

        table_data = []
        for mode in transport_modes:
            table_data.append([mode['_id'], mode['count']])
    
        print(tabulate(table_data, headers=["Transportation Mode", "Count"], tablefmt="fancy_grid"))
        
    def users_in_forbidden_city(self):
        """Task 10: Find users who have tracked activity in the Forbidden City of Beijing."""
        print("Task 10")
        forbidden_city_coords = {"lat": 39.916, "lon": 116.397}
        pipeline = [
            {"$unwind": "$trackpoints"},
            {"$match": {
                "trackpoints.lat": forbidden_city_coords["lat"],
                "trackpoints.lon": forbidden_city_coords["lon"]
            }},
            {"$group": {"_id": "$user_id"}}
        ]
        users = self.db.Activity.aggregate(pipeline)
        print(list(users))

    def task10(self):
        print("\n_________TASK 10: Users with activities in the Forbidden City_________")

        # Query to find users who have done activities in the Forbidden City
        forbidden_city_query = {
            "lat": {"$gte": 39.915, "$lte": 39.917},
            "lon": {"$gte": 116.396, "$lte": 116.398}
        }

        # Aggregation to get distinct user_ids who have trackpoints in Forbidden City
        pipeline = [
            {"$match": forbidden_city_query},  # Match trackpoints within the Forbidden City coordinates
            {"$lookup": {
                "from": "activities",  # Join with activities collection
                "localField": "activity_id",
                "foreignField": "_id",
                "as": "activity_info"
            }},
            {"$unwind": "$activity_info"},  # Unwind the joined activities array
            {"$group": {"_id": "$activity_info.user_id"}},  # Group by user_id to get distinct users
        ]

        # Run the aggregation pipeline
        rows = list(self.db.trackpoints.aggregate(pipeline))

        if rows:
            # Format the output for display
            users = [{"User ID": row["_id"]} for row in rows]
            print(f"Users who have tracked an activity in the Forbidden City:")
            print(tabulate(users, headers="keys", tablefmt="fancy_grid"))
        else:
            print("No users found with activities in the Forbidden City.")
        

    def close(self):
        """Close the database connection."""
        self.connection.close_connection()

In [76]:
program = GeolifeQueries()

You are connected to the database: my_db
-----------------------------------------------



In [61]:
# Part 1. Printing the top ten lines of each table
program.print_tables()

Users
╒══════╤══════════════╕
│   ID │ Has Labels   │
╞══════╪══════════════╡
│  135 │ False        │
├──────┼──────────────┤
│  132 │ False        │
├──────┼──────────────┤
│  104 │ True         │
├──────┼──────────────┤
│  103 │ False        │
├──────┼──────────────┤
│  168 │ False        │
├──────┼──────────────┤
│  157 │ False        │
├──────┼──────────────┤
│  150 │ False        │
├──────┼──────────────┤
│  159 │ False        │
├──────┼──────────────┤
│  166 │ False        │
├──────┼──────────────┤
│  161 │ True         │
╘══════╧══════════════╛


Activities
╒══════════════════════════╤══════════╤═══════════════════════╤═════════════════════╤═════════════════════╕
│ ID                       │   UserID │ Transportation Mode   │ Start Date Time     │ End Date Time       │
╞══════════════════════════╪══════════╪═══════════════════════╪═════════════════════╪═════════════════════╡
│ 671783e2bd276b58f6b2ca82 │      135 │                       │ 2009-01-03 01:21:34 │ 2009-01-03 05:40:31

In [9]:
program.count_users_activities_trackpoints()


Task 1
Users: 182
Activities: 16048
Trackpoints: 9681756


In [13]:
program.average_activities_per_user()

Task 2
Average activities per user: 88.18


In [19]:
program.top_20_users_by_activities()



Task 3
+------+------------------+
|   ID |   Activity Count |
+======+==================+
|  128 |             2102 |
+------+------------------+
|  153 |             1793 |
+------+------------------+
|  025 |              715 |
+------+------------------+
|  163 |              704 |
+------+------------------+
|  062 |              691 |
+------+------------------+
|  144 |              563 |
+------+------------------+
|  041 |              399 |
+------+------------------+
|  085 |              364 |
+------+------------------+
|  004 |              346 |
+------+------------------+
|  140 |              345 |
+------+------------------+
|  167 |              320 |
+------+------------------+
|  068 |              280 |
+------+------------------+
|  017 |              265 |
+------+------------------+
|  003 |              261 |
+------+------------------+
|  014 |              236 |
+------+------------------+
|  126 |              215 |
+------+------------------+
|  030 |     

In [39]:
program.users_who_took_taxi()


Task 4
╒═══════════╕
│   User ID │
╞═══════════╡
│       010 │
├───────────┤
│       058 │
├───────────┤
│       062 │
├───────────┤
│       078 │
├───────────┤
│       080 │
├───────────┤
│       085 │
├───────────┤
│       098 │
├───────────┤
│       111 │
├───────────┤
│       128 │
├───────────┤
│       163 │
╘═══════════╛


In [40]:
# Doesnt work as all transportation modes are ''
program.count_transportation_modes()


Task 5
╒═══════════════════════╤═════════╕
│ Transportation Mode   │   Count │
╞═══════════════════════╪═════════╡
│ walk                  │     480 │
├───────────────────────┼─────────┤
│ car                   │     419 │
├───────────────────────┼─────────┤
│ bike                  │     263 │
├───────────────────────┼─────────┤
│ bus                   │     199 │
├───────────────────────┼─────────┤
│ subway                │     133 │
├───────────────────────┼─────────┤
│ taxi                  │      37 │
├───────────────────────┼─────────┤
│ airplane              │       3 │
├───────────────────────┼─────────┤
│ train                 │       2 │
├───────────────────────┼─────────┤
│ boat                  │       1 │
├───────────────────────┼─────────┤
│ run                   │       1 │
╘═══════════════════════╧═════════╛


In [67]:
program.year_with_most_activities()


Task 6a


OperationFailure: Failed to optimize pipeline :: caused by :: can't convert from BSON type string to Date, full error: {'ok': 0.0, 'errmsg': "Failed to optimize pipeline :: caused by :: can't convert from BSON type string to Date", 'code': 16006, 'codeName': 'Location16006'}

In [77]:
program.task10()


_________TASK 10: Users with activities in the Forbidden City_________
No users found with activities in the Forbidden City.
